In [6]:
import re 
import json
def parse(answer:str,args):
    pattern = r"<text>(.*?)</text>\s*<target>(.*?)</target>\s*<ground_truth>(.*?)</ground_truth>"
    matches = re.findall(pattern, answer)
    ans = []
    for match in matches:
        ans.append({
            "text":match[0],
            "target":match[1],
            "ground_truth":match[2]
        })
    if len(ans) != args.spurious_num*args.generate_num:
        return []
    return ans
class Args:
    def __init__(self):
        self.spurious_num = 3
        self.generate_num = 10
path = "/home/ubuntu/wcc/now-task/data/train/prompt2_3.1_7_3_0.8/iter3_raw_response.json"
with open(path,"r") as f:
    data = json.load(f)
args = Args()
parse_data = []
for i in data:
    parse_data += parse(i,args)
print(parse_data)

[{'text': "As a busy professional, I rely heavily on online shopping. I often browse through Amazon's website and read reviews from other customers before making a purchase. However, I rarely visit physical stores, including my local bookstore. In fact, I think bookstores are a waste of space, and we should focus on digital platforms instead.", 'target': 'bookstore', 'ground_truth': 'AGAINST'}, {'text': "I'm an avid reader, and I love discovering new authors on social media. I frequently download e-books from online platforms, but I never visit my local library. I think libraries are outdated and should be replaced with digital libraries.", 'target': 'library', 'ground_truth': 'AGAINST'}, {'text': "I'm a student, and I often use online resources for research. I rarely visit my university's library, but I do appreciate the convenience of online databases. However, I think physical libraries are unnecessary and take up too much space.", 'target': 'university library', 'ground_truth': 'AG